## Generate Static Graphs

These are the input parameters for the notebook. They will be automatically changed when the scripts to generate monthly statistics are run. You can modify them manually to generate multiple plots locally as well.

Pass in `None` to remove the filters and plot all data. This is not recommended for production settings, but might be useful for reports based on data snapshots.

In [1]:
year = None
month = None
program = "prepilot"
study_type = "program"
mode_of_interest = "e-bike"
include_test_users = False

In [2]:
from collections import defaultdict
import datetime

import numpy as np
import pandas as pd

from plots import *
import scaffolding

sns.set_style("whitegrid")
sns.set()
%matplotlib inline

URL not formatted, defaulting to "Stage_database"
Connecting to database URL db


In [3]:
# Do not run this notebook at all unless it is for a program; nbclient will run up through this cell
if study_type != "program":
    ipython = get_ipython()
    ipython._showtraceback = scaffolding.no_traceback_handler
    raise Exception("The plots in this notebook are only relevant to programs")

In [4]:
# Loading mapping dictionaries from mapping_dictionaries notebook
%store -r dic_re
%store -r dic_pur

# convert a dictionary to a defaultdict
dic_re = defaultdict(lambda: 'Other',dic_re)
dic_pur = defaultdict(lambda: 'Other',dic_pur)

## Collect Data From Database

In [5]:
expanded_ct, file_suffix, quality_text, debug_df = scaffolding.load_viz_notebook_data(year,
                                                                            month,
                                                                            program,
                                                                            study_type,
                                                                            dic_re,
                                                                            dic_pur=dic_pur,
                                                                            include_test_users=include_test_users)

0          4c_AGgrHoywg5gYmyL8
1          4c_Ri332Xj3DKaCrdwl
2          4c_6gzfgbVUODrbTdc_
3          4c_eJEIg2mWQYU3ISXU
4          4c_s7jHZr7a0yQOAa1b
                ...           
239    fc_csu_TMUSjqGBWELwW5mZ
240    fc_n2n__1f8tgZOmeizn0pJ
241    fc_n2n_xV9QUA9nMEVPYq1B
242    fc_csu_OZ_EXMB-w436dbCh
243    fc_n2n_gIVaKBYQtpjA7s5-
Name: user_email, Length: 244, dtype: object

Loaded all confirmed trips of length 135471


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.626885e+09,2021-07-21T10:31:16-06:00,"{'type': 'Point', 'coordinates': [-107.8599774...",60f85d2453f7233da0b1a28c,1.626884e+09,2021-07-21T10:13:12.769000-06:00,"{'type': 'Point', 'coordinates': [-107.8579336...",1083.231000,9413.891182,...,7,21,10,31,16,2,America/Denver,612089e1cb13df1d4d22cfd9,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
1,DwellSegmentationTimeFilter,1.626920e+09,2021-07-21T20:05:41.808000-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",60f8dbb173697eb577aaff42,1.626918e+09,2021-07-21T19:38:56.389000-06:00,"{'type': 'Point', 'coordinates': [-107.7964539...",1605.419000,13124.152280,...,7,21,20,5,41,2,America/Denver,612089e1cb13df1d4d22cfda,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
2,DwellSegmentationTimeFilter,1.626971e+09,2021-07-22T10:15:32.259000-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",60f9aea3f399df93fdc1ab41,1.626969e+09,2021-07-22T09:51:21.159484-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",1451.099516,10587.088615,...,7,22,10,15,32,3,America/Denver,612089e1cb13df1d4d22cfdb,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
3,DwellSegmentationTimeFilter,1.626989e+09,2021-07-22T15:21:31-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",60f9f4f122abd49191e21f5c,1.626988e+09,2021-07-22T15:14:35.056996-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",415.943004,3362.028245,...,7,22,15,21,31,3,America/Denver,612089e1cb13df1d4d22cfdc,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
4,DwellSegmentationTimeFilter,1.626990e+09,2021-07-22T15:32:52.960000-06:00,"{'type': 'Point', 'coordinates': [-107.8601932...",60f9f4f122abd49191e21f5e,1.626989e+09,2021-07-22T15:24:53.289799-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",479.670201,4084.311560,...,7,22,15,32,52,3,America/Denver,612089e2cb13df1d4d22cfdd,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09


After filtering, found 134985 participant trips 


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_month,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts
0,DwellSegmentationTimeFilter,1.626885e+09,2021-07-21T10:31:16-06:00,"{'type': 'Point', 'coordinates': [-107.8599774...",60f85d2453f7233da0b1a28c,1.626884e+09,2021-07-21T10:13:12.769000-06:00,"{'type': 'Point', 'coordinates': [-107.8579336...",1083.231000,9413.891182,...,7,21,10,31,16,2,America/Denver,612089e1cb13df1d4d22cfd9,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
1,DwellSegmentationTimeFilter,1.626920e+09,2021-07-21T20:05:41.808000-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",60f8dbb173697eb577aaff42,1.626918e+09,2021-07-21T19:38:56.389000-06:00,"{'type': 'Point', 'coordinates': [-107.7964539...",1605.419000,13124.152280,...,7,21,20,5,41,2,America/Denver,612089e1cb13df1d4d22cfda,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
2,DwellSegmentationTimeFilter,1.626971e+09,2021-07-22T10:15:32.259000-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",60f9aea3f399df93fdc1ab41,1.626969e+09,2021-07-22T09:51:21.159484-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",1451.099516,10587.088615,...,7,22,10,15,32,3,America/Denver,612089e1cb13df1d4d22cfdb,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
3,DwellSegmentationTimeFilter,1.626989e+09,2021-07-22T15:21:31-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",60f9f4f122abd49191e21f5c,1.626988e+09,2021-07-22T15:14:35.056996-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",415.943004,3362.028245,...,7,22,15,21,31,3,America/Denver,612089e1cb13df1d4d22cfdc,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09
4,DwellSegmentationTimeFilter,1.626990e+09,2021-07-22T15:32:52.960000-06:00,"{'type': 'Point', 'coordinates': [-107.8601932...",60f9f4f122abd49191e21f5e,1.626989e+09,2021-07-22T15:24:53.289799-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",479.670201,4084.311560,...,7,22,15,32,52,3,America/Denver,612089e2cb13df1d4d22cfdd,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09


...

,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_day,end_local_dt_hour,end_local_dt_minute,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts,user_id_str
0,DwellSegmentationTimeFilter,1.626885e+09,2021-07-21T10:31:16-06:00,"{'type': 'Point', 'coordinates': [-107.8599774...",60f85d2453f7233da0b1a28c,1.626884e+09,2021-07-21T10:13:12.769000-06:00,"{'type': 'Point', 'coordinates': [-107.8579336...",1083.231000,9413.891182,...,21,10,31,16,2,America/Denver,612089e1cb13df1d4d22cfd9,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312
1,DwellSegmentationTimeFilter,1.626920e+09,2021-07-21T20:05:41.808000-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",60f8dbb173697eb577aaff42,1.626918e+09,2021-07-21T19:38:56.389000-06:00,"{'type': 'Point', 'coordinates': [-107.7964539...",1605.419000,13124.152280,...,21,20,5,41,2,America/Denver,612089e1cb13df1d4d22cfda,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312
2,DwellSegmentationTimeFilter,1.626971e+09,2021-07-22T10:15:32.259000-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",60f9aea3f399df93fdc1ab41,1.626969e+09,2021-07-22T09:51:21.159484-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",1451.099516,10587.088615,...,22,10,15,32,3,America/Denver,612089e1cb13df1d4d22cfdb,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312
3,DwellSegmentationTimeFilter,1.626989e+09,2021-07-22T15:21:31-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",60f9f4f122abd49191e21f5c,1.626988e+09,2021-07-22T15:14:35.056996-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",415.943004,3362.028245,...,22,15,21,31,3,America/Denver,612089e1cb13df1d4d22cfdc,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312
4,DwellSegmentationTimeFilter,1.626990e+09,2021-07-22T15:32:52.960000-06:00,"{'type': 'Point', 'coordinates': [-107.8601932...",60f9f4f122abd49191e21f5e,1.626989e+09,2021-07-22T15:24:53.289799-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",479.670201,4084.311560,...,22,15,32,52,3,America/Denver,612089e2cb13df1d4d22cfdd,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312


,mode_confirm,purpose_confirm,replaced_mode
0,drove_alone,work,NaN
1,drove_alone,home,NaN
2,drove_alone,work,NaN
3,drove_alone,meal,NaN
4,drove_alone,meal,NaN


After expanding, columns went from 39 -> 42


,source,end_ts,end_fmt_time,end_loc,raw_trip,start_ts,start_fmt_time,start_loc,duration,distance,...,end_local_dt_second,end_local_dt_weekday,end_local_dt_timezone,_id,user_id,metadata_write_ts,user_id_str,mode_confirm,purpose_confirm,replaced_mode
0,DwellSegmentationTimeFilter,1.626885e+09,2021-07-21T10:31:16-06:00,"{'type': 'Point', 'coordinates': [-107.8599774...",60f85d2453f7233da0b1a28c,1.626884e+09,2021-07-21T10:13:12.769000-06:00,"{'type': 'Point', 'coordinates': [-107.8579336...",1083.231000,9413.891182,...,16,2,America/Denver,612089e1cb13df1d4d22cfd9,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,work,NaN
1,DwellSegmentationTimeFilter,1.626920e+09,2021-07-21T20:05:41.808000-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",60f8dbb173697eb577aaff42,1.626918e+09,2021-07-21T19:38:56.389000-06:00,"{'type': 'Point', 'coordinates': [-107.7964539...",1605.419000,13124.152280,...,41,2,America/Denver,612089e1cb13df1d4d22cfda,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,home,NaN
2,DwellSegmentationTimeFilter,1.626971e+09,2021-07-22T10:15:32.259000-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",60f9aea3f399df93fdc1ab41,1.626969e+09,2021-07-22T09:51:21.159484-06:00,"{'type': 'Point', 'coordinates': [-107.8643886...",1451.099516,10587.088615,...,32,3,America/Denver,612089e1cb13df1d4d22cfdb,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,work,NaN
3,DwellSegmentationTimeFilter,1.626989e+09,2021-07-22T15:21:31-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",60f9f4f122abd49191e21f5c,1.626988e+09,2021-07-22T15:14:35.056996-06:00,"{'type': 'Point', 'coordinates': [-107.860199,...",415.943004,3362.028245,...,31,3,America/Denver,612089e1cb13df1d4d22cfdc,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,meal,NaN
4,DwellSegmentationTimeFilter,1.626990e+09,2021-07-22T15:32:52.960000-06:00,"{'type': 'Point', 'coordinates': [-107.8601932...",60f9f4f122abd49191e21f5e,1.626989e+09,2021-07-22T15:24:53.289799-06:00,"{'type': 'Point', 'coordinates': [-107.8743952...",479.670201,4084.311560,...,52,3,America/Denver,612089e2cb13df1d4d22cfdd,e9e479e9-5c3f-4345-a885-dadb7999b312,1.629522e+09,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,meal,NaN


_prepilot
Based on 49952 confirmed trips from 145 users
of 111164 total trips from 148 users (44.94%)


In [ ]:
if 'mode_confirm' in expanded_ct.columns:
    mode_of_interest_df = expanded_ct.query(f"mode_confirm == '{mode_of_interest}'")
    debug_df.loc[f"{mode_of_interest}_trips"] = len(mode_of_interest_df)
    debug_df.loc[f"{mode_of_interest}_trips_with_replaced_mode"] = scaffolding.trip_label_count("Replaced_mode", mode_of_interest_df)

## Data Preprocessing

In [6]:
# Get timestamp from known year/month/day aggregated to days
sel_cols_no_label_dep = ['user_id','start_local_dt_year','start_local_dt_month','start_local_dt_day','distance_miles']
sel_cols_with_label_dep = sel_cols_no_label_dep + ['mode_confirm']
if len(expanded_ct) == 0:
    data = expanded_ct.copy()
elif "mode_confirm" not in expanded_ct.columns:
    data = expanded_ct[sel_cols_no_label_dep].copy()
else:
    data = expanded_ct[sel_cols_with_label_dep].copy()
    
if len(expanded_ct) > 0:
    data.rename(columns={'start_local_dt_year':'year','start_local_dt_month':'month','start_local_dt_day':'day'}, inplace=True)
    data['date_time'] = pd.to_datetime(data[['year','month','day']])
    data = data.drop(columns=['year','month','day'])

    # Categorical type will include all days/modes in groupby even if there is no data for a particular tabulation
    data.user_id = pd.Categorical(data.user_id)
    data.date_time = pd.Categorical(data.date_time)
    
data.head()

,user_id,mode_confirm,distance_miles,date_time
0,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,5.836613,2021-07-21
1,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,8.136974,2021-07-21
2,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,6.563995,2021-07-22
3,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,2.084458,2021-07-22
4,e9e479e9-5c3f-4345-a885-dadb7999b312,drove_alone,2.532273,2021-07-22


In [ ]:
if len(expanded_ct) > 0:
    # Get the count of unique users that were active on each given date
    active_users = pd.DataFrame(data.groupby(['date_time'], as_index=False)['user_id'].nunique())
    active_users.rename(columns={'user_id':'active_users'}, inplace=True)

    if "mode_confirm" in expanded_ct.columns:
        # Count the number of trips for each confirmed mode
        mode_counts = data.groupby(['user_id','date_time','mode_confirm'], as_index=False).size()
        mode_counts.rename(columns={'size':'trip_count'}, inplace=True)

        # Sum daily distance traveled for each mode
        mode_distance = data.groupby(['user_id','date_time','mode_confirm'], as_index=False)[['distance_miles']].sum()
        mode_distance.rename(columns={'sum':'distance_miles'}, inplace=True)
        mode_distance['distance_miles'] = mode_distance['distance_miles'].fillna(0)

        # Add 7-day rolling avg smoothing to better see trends
        mode_counts['trip_count_smooth'] = mode_counts.groupby(['user_id','mode_confirm'])['trip_count'].apply(lambda x: x.rolling(7,1).mean())
        mode_distance['distance_miles_smooth'] = mode_distance.groupby(['user_id','mode_confirm'])['distance_miles'].apply(lambda x: x.rolling(7,1).mean())
        
        # This is the mode specific part
        mode_counts_interest = mode_counts[mode_counts['mode_confirm']==mode_of_interest].copy()
        mode_distance_interest = mode_distance[mode_distance['mode_confirm']==mode_of_interest].copy()
        
        # Mapping new mode labels with dictionaries
        mode_counts['Mode_confirm'] = mode_counts['mode_confirm'].map(dic_re)
        mode_counts_interest['Mode_confirm'] = mode_counts_interest['mode_confirm'].map(dic_re)
        mode_distance_interest['Mode_confirm'] = mode_distance_interest['mode_confirm'].map(dic_re)

In [ ]:
quality_text = scaffolding.get_quality_text(expanded_ct, mode_counts_interest, mode_of_interest, include_test_users)

## Generate Timeseries Plots

### Daily trips for mode of interest

In [ ]:
# Plot of total ebikeshare trips across all users
plot_title_no_quality = f'Daily {mode_of_interest} Trips'
file_name = f"ts_{mode_of_interest}_share%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")
    plot_data = mode_counts_interest.groupby(['date_time'])['trip_count_smooth'].agg(['sum']).reset_index()
    
    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Trip Count'

    timeseries_plot(plot_data['date_time'], plot_data['sum'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Daily mileage for mode of interest

In [ ]:
# Plot of total ebikeshare mileage across all users
plot_title_no_quality = f'Daily {mode_of_interest} Mileage'
file_name = f"ts_{mode_of_interest}_miles%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data = mode_distance_interest.groupby(['date_time'])['distance_miles'].agg(['sum']).reset_index()

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Miles'

    timeseries_plot(plot_data['date_time'], plot_data['sum'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Daily mileage per user for mode of interest

In [ ]:
# Plot of total ebikeshare mileage normalized by number of users
plot_title_no_quality = f'Daily {mode_of_interest} Mileage per Active User'
file_name = f"ts_{mode_of_interest}_miles_user%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data = mode_distance_interest.groupby(['date_time'])['distance_miles'].agg(['sum']).reset_index()
    plot_data = plot_data.merge(active_users, on='date_time')
    plot_data['mileage_per_user'] = plot_data['sum'] / plot_data['active_users']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'miles/user'

    timeseries_plot(plot_data['date_time'], plot_data['mileage_per_user'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Proportion of total daily mileage for mode of interest

In [ ]:
# Plot of ebike mileage share proportion across all users
plot_title_no_quality = f'Daily {mode_of_interest} Mileage Proportion (All Users, excluding air)'
file_name = f"ts_{mode_of_interest}_miles_proportion%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data = mode_distance_interest.groupby(['date_time'], as_index=False)['distance_miles_smooth'].sum()
    total_miles = mode_distance.groupby(['date_time'], as_index=False)['distance_miles_smooth'].sum()
    plot_data = plot_data.merge(total_miles, on=['date_time'])
    plot_data['miles_proportion'] = plot_data['distance_miles_smooth_x'] / plot_data['distance_miles_smooth_y']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Proportion of Daily Miles'

    timeseries_plot(plot_data['date_time'], plot_data['miles_proportion'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data, file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Proportion of total daily trips for mode of interest

In [ ]:
# Plot of ebike trip share proportion across all users
plot_title_no_quality = f'Daily {mode_of_interest} Trip Proportion (All Users, excluding air)'
file_name = f"ts_{mode_of_interest}_trips_proportion%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data = mode_counts.groupby(['date_time','mode_confirm'], as_index=False)['trip_count_smooth'].sum()
    total_trips = plot_data.groupby(['date_time'], as_index=False).sum()
    plot_data = plot_data.merge(total_trips, on='date_time')
    plot_data['trip_proportion'] = plot_data['trip_count_smooth_x'] / plot_data['trip_count_smooth_y']
    plot_data = plot_data[plot_data['mode_confirm']==mode_of_interest]

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Proportion of Daily Trips'

    timeseries_plot(plot_data['date_time'], plot_data['trip_proportion'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data.drop(columns=['mode_confirm','trip_count_smooth_x','trip_count_smooth_y']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Daily trip proportion for mode of interest with error bounds

In [ ]:
# Plots the number of trips per user with error bars
# from the seaborn documentation:
# By default, the plot aggregates over multiple y values at each value of x and 
# shows an estimate of the central tendency and a confidence interval for that estimate.
# In our case, we have multiple trip proportions (one per user) for each day
# so the band represents the variation of the number of trips and the thick line represents the mean/median (unsure which)
# but this still doesn't tell us which users have dropped their ridership

plot_title_no_quality = f'Daily {mode_of_interest} Trip Proportion for Individual users (Running average and variation)'
file_name = f"ts_{mode_of_interest}_trip_individual_variation%s"%file_suffix

try:
    if len(mode_counts_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data_mode = mode_counts_interest.groupby(['date_time', 'user_id'], as_index=False)["trip_count_smooth"].sum()
    plot_data = mode_counts.groupby(['date_time','user_id'], as_index=False)['trip_count_smooth'].sum()
    plot_data = plot_data.merge(plot_data_mode, on=['date_time', 'user_id'])
    plot_data['mode_trip_proportion'] = plot_data['trip_count_smooth_y'] / plot_data['trip_count_smooth_x']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Proportion of Daily Trips'

    timeseries_plot(plot_data['date_time'], plot_data['mode_trip_proportion'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data.drop(columns=['user_id','trip_count_smooth_x','trip_count_smooth_y']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)

### Daily mileage proportion for mode of interest with error bounds

In [ ]:
# Plots the number of trips per user with error bars
# from the seaborn documentation:
# By default, the plot aggregates over multiple y values at each value of x and 
# shows an estimate of the central tendency and a confidence interval for that estimate.
# In our case, we have multiple trip proportions (one per user) for each day
# so the band represents the variation of the number of trips and the thick line represents the mean/median (unsure which)
# but this still doesn't tell us which users have dropped their ridership

plot_title = f'Daily {mode_of_interest} Mileage Proportion for Individual users (Running average and variation)'
file_name = f"ts_{mode_of_interest}_mile_individual_variation%s"%file_suffix

try:
    if len(mode_distance_interest) == 0:
        # force error generation so that we will go into the "missing" data code path
        raise RuntimeError(f"No {mode_of_interest} trips found")

    plot_data_mode = mode_distance_interest.groupby(['date_time', 'user_id'], as_index=False)["distance_miles_smooth"].sum()
    plot_data = mode_distance.groupby(['date_time','user_id'], as_index=False)['distance_miles_smooth'].sum()
    plot_data = plot_data.merge(plot_data_mode, on=['date_time', 'user_id'])
    plot_data['mode_miles_proportion'] = plot_data['distance_miles_smooth_y'] / plot_data['distance_miles_smooth_x']

    plot_title= plot_title_no_quality+"\n"+quality_text
    ylab = 'Proportion of Daily Miles'

    timeseries_plot(plot_data['date_time'], plot_data['mode_miles_proportion'], plot_title, ylab, file_name)
    alt_text = store_alt_text_timeseries(plot_data.drop(columns=['user_id','distance_miles_smooth_x','distance_miles_smooth_y']), file_name, plot_title)
except:
    generate_missing_plot(plot_title_no_quality,debug_df,file_name)
    alt_text = store_alt_text_missing(debug_df, file_name, plot_title_no_quality)